# McDonald's Card Data Analysis - v3
## March 2018
### Dr Jose M Albornoz

# 1.- Import necessary modules, define SQLContext

In [1]:
# Import required modules
from pyspark.context import SparkContext
from pyspark.sql.context import SQLContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import unix_timestamp

In [2]:
# Define SQLContext
sqlContext = SQLContext(sc)

# 2.- Generic function to load data from a csv file

In [3]:
def load_data(filename, schema, columns = None):
    df = sqlContext.read.format('com.databricks.spark.csv').option("delimiter", ";").options(header='false'). \
    load(filename, schema = schema)
    if columns is None:
        # If no columns are given, then select all
        columns = schema.names
    return df.select(columns)

# 3.- Data schema

In [4]:
schema = StructType([ 
    StructField('store_number', IntegerType(), True), 
    StructField('terminal_number', IntegerType(), True), 
    StructField('transaction_date', StringType(), True), 
    StructField('transaction_time', IntegerType(), True), 
    StructField('transaction_amount', IntegerType(), True),
    StructField('card_scheme', StringType(), True),
    StructField('pan_token', StringType(), True),
    StructField('empty_field', IntegerType(), True)    
])

# 4.- Load data

In [5]:
filename = 'McD_Card_Data/CT_201709_p1.csv'
df_p1 = load_data(filename, schema)

In [6]:
filename = 'McD_Card_Data/CT_201709_p2.csv'
df_p2 = load_data(filename, schema)

In [7]:
filename = 'McD_Card_Data/CT_201709_p3.csv'
df_p3 = load_data(filename, schema)

In [8]:
filename = 'McD_Card_Data/CT_201710_p1.csv'
df_p4 = load_data(filename, schema)

In [9]:
filename = 'McD_Card_Data/CT_201710_p2.csv'
df_p5 = load_data(filename, schema)

In [10]:
filename = 'McD_Card_Data/CT_201710_p3.csv'
df_p6 = load_data(filename, schema)

In [11]:
filename = 'McD_Card_Data/CT_201711_p1.csv'
df_p7 = load_data(filename, schema)

In [12]:
filename = 'McD_Card_Data/CT_201711_p2.csv'
df_p8 = load_data(filename, schema)

In [13]:
filename = 'McD_Card_Data/CT_201711_p3.csv'
df_p9 = load_data(filename, schema)

In [14]:
# filename = 'McD_Card_Data/CT_201712_p1.csv'
# df_p10 = load_data(filename, schema)

In [15]:
# filename = 'McD_Card_Data/CT_201712_p2.csv'
# df_p11 = load_data(filename, schema)

In [16]:
# filename = 'McD_Card_Data/CT_201712_p3.csv'
# df_p12 = load_data(filename, schema)

# 5.- Concatenate data

In [17]:
df_data = df_p1.unionAll(df_p2)

In [18]:
df_data = df_data.unionAll(df_p3)

In [19]:
df_data = df_data.unionAll(df_p4)

In [20]:
df_data = df_data.unionAll(df_p5)

In [21]:
df_data = df_data.unionAll(df_p6)

In [22]:
df_data = df_data.unionAll(df_p7)

In [23]:
df_data = df_data.unionAll(df_p8)

In [24]:
df_data = df_data.unionAll(df_p9)

In [25]:
# df_data = df_data.unionAll(df_p10)

In [26]:
# df_data = df_data.unionAll(df_p11)

In [27]:
# df_data = df_data.unionAll(df_p12)

In [28]:
df_data.count()

89421024

# 6.- Remove unnecessary data from memory

In [29]:
df_p1.unpersist()
df_p2.unpersist()
df_p3.unpersist()
df_p4.unpersist()
df_p5.unpersist()
df_p6.unpersist()
df_p7.unpersist()
df_p8.unpersist()
df_p9.unpersist()
# df_p10.unpersist()
# df_p11.unpersist()
# df_p12.unpersist()

DataFrame[store_number: int, terminal_number: int, transaction_date: string, transaction_time: int, transaction_amount: int, card_scheme: string, pan_token: string, empty_field: int]

# 7.- Register data as table

In [30]:
df_data.registerTempTable("data")

# 8.- Identify unique cards, register as table

In [31]:
df_unique_cards = sqlContext.sql("SELECT DISTINCT pan_token FROM data")

In [32]:
df_unique_cards.count()

26994812

In [33]:
#df_unique_cards.registerTempTable("unique_cards")

# 9.- Order data by transaction date and card token

In [32]:
df_data = sqlContext.sql("SELECT * FROM data ORDER BY pan_token, transaction_date, transaction_time")

In [35]:
df_data.show(40)

+------------+---------------+----------------+----------------+------------------+-----------+--------------------+-----------+
|store_number|terminal_number|transaction_date|transaction_time|transaction_amount|card_scheme|           pan_token|empty_field|
+------------+---------------+----------------+----------------+------------------+-----------+--------------------+-----------+
|         751|             20|      2017/09/09|            1242|              1755|          S|0000000ABF375473C...|       null|
|         930|             20|      2017/11/11|            2348|              1307|          S|0000000ABF375473C...|       null|
|         719|             20|      2017/10/28|            2058|               547|          S|0000007FCA6A76508...|       null|
|           7|             20|      2017/09/24|              26|               778|          S|000001195509842BB...|       null|
|           7|              2|      2017/10/01|            2310|               838|          S|00

# 10.- Find frequency of visits per customer

## 10.1.- Number of unique customers that have visited at least twice in any calendar week (H) 

In [33]:
df_weekly1 = df_data.groupBy("pan_token", weekofyear(from_unixtime(unix_timestamp('transaction_date', 'yyy/MM/dd'))).\
                            alias('week')).agg(count("*").alias('visits')).sort('pan_token')
#df_weekly1.show(20)

In [34]:
df_weekly1.registerTempTable("weekly_visit1")

In [35]:
df_weekly2 = sqlContext.sql("SELECT DISTINCT pan_token, visits \
                             FROM weekly_visit1 \
                             WHERE visits >= 2")

In [36]:
#df_weekly2.show(20)

In [37]:
df_weekly2.registerTempTable("weekly_visit2")

In [38]:
df_weekly3 = sqlContext.sql("SELECT DISTINCT pan_token \
                             FROM weekly_visit2")

In [39]:
#df_weekly3.show(20)

In [40]:
H_count = df_weekly3.count()
H_count

7695617

In [41]:
df_weekly1.unpersist()
df_weekly2.unpersist()
df_weekly3.unpersist()

DataFrame[pan_token: string]

## 10.2.- Number of unique customers that visit twice in any calendar month (M)

In [42]:
df_monthly1 = df_data.groupBy("pan_token", month(from_unixtime(unix_timestamp('transaction_date', 'yyy/MM/dd'))).\
                            alias('month')).agg(count("*").alias('visits')).sort('pan_token')
#df_monthly1.show(20)

In [43]:
df_monthly1.registerTempTable("monthly_visit1")

In [44]:
df_monthly2 = sqlContext.sql("SELECT DISTINCT pan_token, visits \
                             FROM monthly_visit1 \
                             WHERE visits = 1")

In [45]:
M_count = df_monthly2.count()
M_count

19786649

In [46]:
df_monthly1.unpersist()
df_monthly2.unpersist()

DataFrame[pan_token: string, visits: bigint]

## 10.3.- Number of unique customers that visit once in any 6 calendar weeks lapse (L)

In [47]:
interval = 42 * 24 * 60 * 60     # 6 weeks = 42 days  
gdf = df_data.\
withColumn('6week_interval', \
           from_unixtime(floor(unix_timestamp('transaction_date', 'yyy/MM/dd') / interval) * interval\
           + ((unix_timestamp(lit('2017/09/01'),'yyy/MM/dd')/interval)%1)*interval))

df_6weekly1 = gdf.groupBy('pan_token', '6week_interval').agg(count("*").alias('visits')).sort('pan_token')

#df_6weekly.show(20)

In [48]:
df_6weekly1.registerTempTable("6weekly_visit1")

In [49]:
df_6weekly2 = sqlContext.sql("SELECT DISTINCT pan_token, visits \
                             FROM 6weekly_visit1 \
                             WHERE visits = 1")

In [50]:
#df_6weekly2.show(40)

In [51]:
L_count = df_6weekly2.count()
L_count

19560960

In [52]:
gdf.unpersist()
df_6weekly1.unpersist()
df_6weekly2.unpersist()

DataFrame[pan_token: string, visits: bigint]

## 10.4.- Number of unique customers that visit once in a 3 calendar month period (I)

In [53]:
interval = 90 * 24 * 60 * 60     # 3 months = 90 days  
gdf = df_data.\
withColumn('quarter_interval', \
           from_unixtime(floor(unix_timestamp('transaction_date', 'yyy/MM/dd') / interval) * interval\
           + ((unix_timestamp(lit('2017/09/01'),'yyy/MM/dd')/interval)%1)*interval))

df_3month1 = gdf.groupBy('pan_token', 'quarter_interval').agg(count("*").alias('visits')).sort('pan_token')

In [54]:
df_3month1.registerTempTable("3month_visit1")

In [55]:
df_3month2 = sqlContext.sql("SELECT DISTINCT pan_token, visits \
                             FROM 3month_visit1 \
                             WHERE visits = 1")

In [56]:
I_count = df_3month2.count()
I_count

16702547